In [38]:
%matplotlib notebook
#Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, probplot
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display


In [63]:
#Carregando o dataset principal (target) sobre a produção de carne
target_carne_produzida = pd.read_csv('meat-production-tonnes.csv')

#Renomeando e criando algumas variáveis do dataset para deixar mais organizado e separar apenas os dados de 2010
pais_carne_produzida = target_carne_produzida['Entity']    
producao_2010 = target_carne_produzida['Year'] == 2010
target_carne_produzida = target_carne_produzida.loc[producao_2010, :]
target_carne_produzida = target_carne_produzida.rename(columns={'Entity' : 'Country'})
target_carne_produzida = target_carne_produzida.rename(columns={'Livestock Primary - Meat, Total - 1765 - Production - 5510 - tonnes (tonnes)' : 'Meat Tonnes'})

In [64]:
#Carregando um dataset de feature sobre o consumo de carne
feature_consumo_per_capita = pd.read_csv('per-capita-meat-consumption-by-type-kilograms-per-year.csv')

#Renomeando e criando algumas variáveis do dataset para deixar mais organizado e separar apenas os dados de 2010
producao_2010 = feature_consumo_per_capita['Year'] == 2010
consumo_2010_per_cpt = feature_consumo_per_capita.loc[producao_2010, :]
consumo_2010_per_cpt = consumo_2010_per_cpt.loc[:, ['Entity', 'Year', 'Beef and bufallo (kilograms per year)']]
consumo_2010_per_cpt = consumo_2010_per_cpt.rename(columns={'Entity' : 'Country'})
pais_consumo_2010_per_cpt = consumo_2010_per_cpt['Country']

In [65]:
#Carregando um dataset de features importantes (IDH, Expect. de vida, etc...)
feature_IDH = pd.read_csv('HDI.csv')

#Renomeando e criando algumas variáveis do dataset para deixar mais organizado
IDH_dados_importantes = feature_IDH.loc[:,['Country', 'HDI', 'Life expectancy', 'Mean years of schooling', 'Coefficient of human inequality']]
pais_IDH = IDH_dados_importantes['Country']

In [66]:
#Carregando um dataset de feature sobre o desemprego 
feature_desemprego_pais = pd.read_csv('API_ILO_country_YU.csv')

#Renomeando e criando algumas variáveis do dataset para deixar mais organizado e separar apenas os dados de 2010
desemprego_2010 = feature_desemprego_pais.loc[:,['Country Name', '2010']]
desemprego_2010 = desemprego_2010.rename(columns={'Country Name' : 'Country'})
desemprego_2010 = desemprego_2010.rename(columns={'2010' : 'Desemprego em 2010'})

In [70]:
#Carregando um dataset de feature sobre o PIB
feature_pib = pd.read_csv('GDP.csv')

#Renomeando e criando algumas variáveis do dataset para deixar mais organizado e separar apenas os dados de 2010
pib_2010 = feature_pib.loc[:,['Country ', '2010']]
pib_2010 = pib_2010.rename(columns={'2010' : 'PIB em 2010'})

In [71]:
feature_dados_commodity = pd.read_csv('commodity_trade_statistics_data.csv') 
ano_2010 = feature_dados_commodity['year'] == 2010
commodity_2010 = feature_dados_commodity.loc[ano_2010, :]
commodity_2010 = commodity_2010.loc[:, ['country_or_area', 'year', 'commodity', 'flow', 'trade_usd', 'weight_kg', 'category']]

FileNotFoundError: [Errno 2] No such file or directory: 'commodity_trade_statistics_data.csv'

In [75]:
#Juntando todos os datasets em um único, tendo como base (index principal) o país
tabela_1 = target_carne_produzida.set_index('Country').join(consumo_2010_per_cpt.set_index('Country'), lsuffix='_carne', rsuffix='_consumo')
tabela_2 = tabela_1.join(IDH_dados_importantes.set_index('Country'), lsuffix='_carne', rsuffix='_consumo')
tabela_3 = tabela_2.join(desemprego_2010.set_index('Country'), lsuffix='_carne', rsuffix='_consumo' )
tabela_final = tabela_3.join(pib_2010.set_index('Country '), lsuffix='_carne', rsuffix='_consumo')

In [76]:
#Retirando os países que apresentarem 'NaN' e removendo algumas colunas para deixar mais organizado e melhor para ser analisado
tabela_final = tabela_final.dropna()
tabela_final = tabela_final.drop(columns=['Code', 'Year_consumo', 'Year_carne'])
tabela_final

,Meat Tonnes,Beef and bufallo (kilograms per year),HDI,Life expectancy,Mean years of schooling,Coefficient of human inequality,Desemprego em 2010,PIB em 2010
Country,,,,,,,,
Afghanistan,328160,4.70,0.479,60.7,3.6,30.4,20.600000,1637.377987
Albania,91440,21.38,0.764,78.0,9.6,13.4,25.799999,9628.025783
Angola,217027,6.67,0.533,52.7,5.0,36.6,10.800000,6230.297028
Argentina,4721922,54.65,0.827,76.5,9.9,15.2,19.500000,18524.856760
Armenia,69484,18.01,0.743,74.9,11.3,9.2,38.299999,6565.678660
...,...,...,...,...,...,...,...,...
United States,41952752,38.66,0.920,79.2,13.2,12.9,18.700001,48466.823380
Uruguay,652983,30.76,0.795,77.4,8.6,15.4,20.799999,16808.284880
Uzbekistan,818300,23.98,0.701,69.4,12.0,15.3,20.600000,5136.234068


In [79]:
#Renomeando todas as colunas para deixar em português e com algumas unidades utilizadas
tabela_final = tabela_final.rename(columns={'Beef and bufallo (kilograms per year)' : 'Carne consumida/capita (kg/ano)'})
tabela_final = tabela_final.rename(columns={'HDI' : 'IDH'})
tabela_final = tabela_final.rename(columns={'Meat Tonnes' : 'Carne produzida (toneladas/ano)'})
tabela_final = tabela_final.rename(columns={'Life expectancy' : 'Expectativa de vida (anos)'})
tabela_final = tabela_final.rename(columns={'Mean years of schooling' : 'Média de escolaridade (anos)'})
tabela_final = tabela_final.rename(columns={'Coefficient of human inequality' : 'Coeficiente de desigualdade humana'})
tabela_final = tabela_final.rename(columns={'Desemprego em 2010' : 'Taxa de desemprego (%)'})
tabela_final = tabela_final.rename(columns={'PIB em 2010' : 'PIB per capita (U$)'})

In [80]:
tabela_final.head(50)

,Carne produzida (toneladas/ano),Carne consumida/capita (kg/ano),IDH,Expectativa de vida (anos),Média de escolaridade (anos),Coeficiente de desigualdade humana,Taxa de desemprego (%),PIB per capita (U$)
Country,,,,,,,,
Afghanistan,328160,4.70,0.479,60.7,3.6,30.4,20.600000,1637.377987
Albania,91440,21.38,0.764,78.0,9.6,13.4,25.799999,9628.025783
Angola,217027,6.67,0.533,52.7,5.0,36.6,10.800000,6230.297028
Argentina,4721922,54.65,0.827,76.5,9.9,15.2,19.500000,18524.856760
Armenia,69484,18.01,0.743,74.9,11.3,9.2,38.299999,6565.678660
Australia,4047952,37.25,0.939,82.5,13.2,8.0,11.400000,39324.368950
Austria,920059,17.41,0.893,81.6,11.3,8.5,8.800000,42006.046880
Azerbaijan,251911,12.35,0.759,70.9,11.2,13.0,14.600000,15884.044790
Bangladesh,607355,1.29,0.579,72.0,5.2,28.6,8.200000,2466.902949


In [81]:
#Criando uma função que sirva para criar as regressões que vamos utilizar para analisar o projeto
def regress(Y,X):
    '''
    Y: coluna do DataFrame utilizada como variável resposta (TARGET)
    X: coluna(s) do DataFrame utilizadas como variável(is) explicativas (FEATURES)
    '''
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    
    return results

In [84]:
#Criando uma nova variável com o dataset final para facilitar
df = tabela_final[['Carne produzida (toneladas/ano)', 'Carne consumida/capita (kg/ano)',
       'IDH', 'Expectativa de vida (anos)', 'Média de escolaridade (anos)',
       'Coeficiente de desigualdade humana', 'Taxa de desemprego (%)',
       'PIB per capita (U$)']]

In [85]:
df.describe()

,Carne produzida (toneladas/ano),Carne consumida/capita (kg/ano),IDH,Expectativa de vida (anos),Média de escolaridade (anos),Coeficiente de desigualdade humana,Taxa de desemprego (%),PIB per capita (U$)
count,1.210000e+02,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000
mean,1.500236e+06,11.749174,0.705091,71.523967,8.559504,19.768595,18.919835,15462.403182
std,4.511072e+06,9.517539,0.162174,8.735770,3.253449,10.302096,11.503569,15349.107505
min,8.380000e+02,0.490000,0.352000,50.100000,1.400000,5.300000,0.700000,795.682826
25%,9.584700e+04,4.990000,0.563000,65.100000,6.100000,10.200000,10.300000,3269.794329
50%,2.858190e+05,8.490000,0.739000,74.200000,9.000000,18.200000,17.100000,10227.325960
75%,9.714000e+05,17.750000,0.843000,78.000000,11.300000,28.600000,25.000000,21048.330520
max,4.195275e+07,54.650000,0.949000,83.700000,13.400000,43.100000,57.200001,85613.586610


In [86]:
#Separando em 'target' e 'feature' para fazer a regressão
X = df[['Carne consumida/capita (kg/ano)',
       'IDH', 'Expectativa de vida (anos)', 'Média de escolaridade (anos)',
       'Coeficiente de desigualdade humana', 'Taxa de desemprego (%)',
       'PIB per capita (U$)']]
Y = df['Carne produzida (toneladas/ano)']

In [87]:
#Fazendo a regressão
results = regress(Y,X)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                  
===========================================================================================
Dep. Variable:     Carne produzida (toneladas/ano)   R-squared:                       0.221
Model:                                         OLS   Adj. R-squared:                  0.173
Method:                              Least Squares   F-statistic:                     4.584
Date:                             Wed, 24 Nov 2021   Prob (F-statistic):           0.000154
Time:                                     08:11:25   Log-Likelihood:                -2010.0
No. Observations:                              121   AIC:                             4036.
Df Residuals:                                  113   BIC:                             4058.
Df Model:                                        7                                         
Covariance Type:                         nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                               -9.24e+06   8.48e+06     -1.089      0.278    -2.6e+07    7.57e+06
Carne consumida/capita (kg/ano)      1.95e+05   5.04e+04      3.872      0.000    9.52e+04    2.95e+05
IDH                                 8.821e+06   1.32e+07      0.668      0.506   -1.74e+07     3.5e+07
Expectativa de vida (anos)         -2.066e+04   1.39e+05     -0.148      0.882   -2.96e+05    2.55e+05
Média de escolaridade (anos)        7.769e+04   4.26e+05      0.182      0.856   -7.67e+05    9.22e+05
Coeficiente de desigualdade humana  1.659e+05   9.93e+04      1.671      0.097   -3.08e+04    3.63e+05
Taxa de desemprego (%)             -1.875e+04   3.66e+04     -0.512      0.610   -9.13e+04    5.38e+04
PIB per capita (U$)                    7.4968     46.782      0.160      0.873     -85.186     100.180
==============================================================================
Omnibus:                      178.950   Durbin-Watson:                   2.091
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10400.199
Skew:                           5.680   Prob(JB):                         0.00
Kurtosis:                      46.975   Cond. No.                     7.83e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.83e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""